In [1]:
import glob
import json
import ipaddress

In [2]:
def read_hosts(path):
    nws = []
    for h in glob.glob(path):
        with open(h) as fp:
            content = json.load(fp)
            nws.append(ipaddress.ip_network(content['hostInterfaces']['eth0']['prefix'], strict=False))
    return nws

In [12]:
def find_in_lines(needle, search):
    diff_specs = []
    for i, l in enumerate(search.splitlines() if type(search) is str else search):
        # print(needle.__str__())
        if needle.__str__() in l:
            diff_specs.append(l)
            # print(f"Found {needle} in line {i}:", l)
    return diff_specs

In [24]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/uscarrier-ospf"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
print("network_name", "specs#","kept#","false_positive#","false_negative#","#false_negative_fakehost","#false_positive_fakehost","#hosts", sep='\t')
for nw in networks:
    nw_name = nw.split("/")[-1]
    # if "strawman" in nw_name:
    #     continue
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
    host_pfxs_count = len(read_hosts(nw + "/hosts/host*.json"))
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv", 'r') as fp:
        nw_content = fp.read().__str__()
    with open(NETWORKS_DIR + f"/origin/policies.csv", 'r') as fp:
        origin_content = fp.read().__str__()
    
    origin_policies = set(origin_content.splitlines()[1:])
    nw_policies = set(nw_content.splitlines()[1:])
    fpos = nw_policies - origin_policies
    fneg = origin_policies - nw_policies
    kept = origin_policies & nw_policies

    # read host
    fneg_host_count = 0
    fpos_host_count = 0
    for pfx in orig_host_prefixes:
        diff_specs = find_in_lines(pfx, list(fneg))
        fneg_host_count += len(diff_specs)
        diff_specs = find_in_lines(pfx, list(fpos))
        fpos_host_count += len(diff_specs)

    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(kept), len(fpos), len(fneg), fneg_host_count, fpos_host_count, host_pfxs_count, sep="\t")
    # break

network_name	specs#	kept#	false_positive#	false_negative#	#false_negative_fakehost	#false_positive_fakehost	#hosts
origin	462739	462739	0	0	0	0	55
anonym-network-k2	549945	428519	121426	34220	8148	1704	110
anonym-network-k6	430751	273659	157092	189080	48582	17882	110
anonym-network-k10	359407	217487	141920	245252	62361	17334	110
anonym-network-k6-2-w-noise-n	430751	273659	157092	189080	48582	17882	110
strawman-1	549945	428519	121426	34220	8148	1704	110
anonym-network-k6-2-w-noise	429344	272835	156509	189904	48699	17720	110


In [ ]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/bics-ospf"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
for nw in networks:
    nw_name = nw.split("/")[-1]
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
    host_pfxs_count = len(read_hosts(nw + "/hosts/host*.json"))
    # read diff
    with open(NETWORKS_DIR + f"/origin/not-in-{nw_name}.csv", 'r') as fp:
        diff_host_count = 0
        content = fp.read().__str__()
        for pfx in orig_host_prefixes:
            diff_specs = find_in_lines(pfx, content)
            diff_host_count += len(diff_specs)
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(content.splitlines()), diff_host_count, host_pfxs_count, sep="\t")
    # break